In [0]:
dbutils.fs.put('/Volumes/workspace/default/emp/emp_prac.csv',"""E_CODE|D_CODE|E_NAME|E_LOC|E_SALARY
E1|D1|Mi3ke|TEXAS|5000
E2|D3|T2om|CHICAGO|6000
E3|D2|1Wayn3e|INDIANAPOLIS|3000
E4|D2|2Joh3n|NEW YORK|4000
E5|D3|Phillip|CHICAGO|7000
E6|D1|Kevin4|SAN JOSE|15000
E7|D1|Be3n|DENVER|3500
E8|D3|Cri5sti7na8|CHICAGO|6500
E9|D1|Stev8e|TEXAS|8500
E10|D2|Ca3rol|INDIANAPOLIS|5500""")

In [0]:
df_emp = spark.read.format('csv').option("header", True).option("inferSchema", True).option("delimiter", '|').load('/Volumes/workspace/default/emp/emp_prac.csv')
display(df_emp)

In [0]:
dbutils.fs.put('/Volumes/workspace/default/emp/emp_prac_currupt.csv',"""E_CODE|D_CODE|E_NAME|E_LOC|E_SALARY|BAREC
E1|D1|Mi3ke|TEXAS|5000
E2|D3|T2om|CHICAGO|6000
E3|D2|1Wayn3e|INDIANAPOLIS|3000
E4|D2|2Joh3n|NEW YORK|4000
E5|D3|Phillip|CHICAGO|7000
E6 D1 Kevin4|SAN JOSE|15000|
E7|D1|Be3n|DENVER|3500
E8|D3|Cri5sti7na8|CHICAGO|6500
E9|D1|Stev8e|TEXAS|8500
E10|D2|Ca3rol|INDIANAPOLIS|5500""")

In [0]:
display(dbutils.fs.ls('/Volumes/workspace/default/emp/emp_prac.csv'))

**PERMISSIVE**

In [0]:
#PERMISSIVE mode let bad records to read


In [0]:
df_emp_currupt =  spark.read.format('csv').option("header",True).option("inferSchema",True).option("delimiter",'|').option("mode","PERMISSIVE").load("/Volumes/workspace/default/emp/emp_prac_currupt.csv")
df_emp_currupt.display()

**FAILFAST**

In [0]:
#Failfast mode is used when the bad records are comming. and we don't want to load need to fail the job we can use it

In [0]:
df_emp_fail =  spark.read.format('csv').option("header",True).option("inferSchema",True).option("delimiter",'|').option("mode","FAILFAST").load("/Volumes/workspace/default/emp/emp_prac_currupt.csv")
df_emp_fail.display()

**DROPMALFORMED**

#dropmalformed load is used whenever the bad recorda arrives reject the old records aand allow onlr good records.

In [0]:
df_emp_malformed = spark.read.format('csv').option("header", True).option("inferSchema", True).option("delimiter", '|').option("mode", "DROPMALFORMED").load('/Volumes/workspace/default/emp/emp_prac_currupt.csv')
display(df_emp_malformed)

To capture the bad record we need specify the path that where the bad records has to be store

In [0]:
df_emp_bad = spark.read.format('csv').option("header", True).option("delimiter", '|').option("badRecordsPath",'/Volumes/workspace/default/emp/bad/emp_prac_bad.csv').load('/Volumes/workspace/default/emp/emp_prac_currupt.csv')
display(df_emp_bad)

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

schema = StructType([
    StructField("column1", StringType(), True),
    StructField("column2", IntegerType(), True),
    StructField("column3", StringType(), True)
])

df_emp_bad_c = spark.read.format('csv').option("header", True).option("delimiter", '|').schema(schema).load('/Volumes/workspace/default/emp/bad/emp_prac_bad.csv')
display(df_emp_bad_c)

In [0]:
df_emp_bad_c = spark.read.format('csv').option("header", True).option("inferSchema", True).option("delimiter", '|').load('/Volumes/workspace/default/emp/bad/emp_prac_bad.csv')
display(df_emp_bad_c)

In [0]:
%fs ls '/Volumes/workspace/default/emp/bad/emp_prac_bad.csv'

In [0]:
df_bad_json = spark.read.option("inferSchema",True).json("/Volumes/workspace/default/emp/bad/emp_prac_bad.csv")
df_bad_json.display()

In [0]:
dbutils.fs.put('/Volumes/workspace/default/emp/badrec/emp_cor_bad_new.csv',"""E_CODE|D_CODE|E_NAME|E_LOC|E_SALARY|BAREC
E1|D1|Mi3ke|TEXAS|5000
E2|D3|T2om|CHICAGO|6000
E3|D2|1Wayn3e|INDIANAPOLIS|3000
E4|D2|2Joh3n|NEW YORK|4000
E5|D3|Phillip|CHICAGO|7000
E6 D1 Kevin4|SAN JOSE|15000|
E7|D1|Be3n|DENVER|3500
E8|D3|Cri5sti7na8|CHICAGO|6500
E9|D1|Stev8e|TEXAS|8500
E10|D2|Ca3rol|INDIANAPOLIS|5500""")

In [0]:
df_bad_cor = spark.read.format("csv").option("header",True).option("inferSchema",True).option("delimiter",'|').option("mode","PERMISSIVE").option("columnNameOfCorruptRecord","BADREC").load('/Volumes/workspace/default/emp/badrec/emp_cor_bad_new.csv')
display(df_bad_cor)

In [0]:
df_bad_cor = spark.read.format("csv").option("header",True).option("inferSchema",True).option("delimiter",'|').option("mode","DROPMALFORMED").option("columnNameOfCorruptRecord","BADREC").load('/Volumes/workspace/default/emp/badrec/emp_cor_bad_new.csv')
display(df_bad_cor)

In [0]:
good_data = df_bad_cor.filter(df_bad_cor.BADREC.isNull())
good_data.write.format("json").mode("overwrite").save("/Volumes/workspace/default/emp/emp_prac_cor_bad.csv")

In [0]:
df_bad_cor = spark.read.format("json").option("mode","PERMISSIVE").option("badRecordsPath",'/Volumes/workspace/default/emp/bad/emp_correct_bcol.csv').option("columnNameOfCorruptRecord","BADREC").load('/Volumes/workspace/default/emp/bad/emp_cor_bad_new.csv')
display(df_bad_cor)

In [0]:
df_emp = spark.read.format('csv').option("header", True).option("inferSchema", True).option("delimiter", '|').load('/Volumes/workspace/default/emp/emp_prac.csv')
display(df_emp)

In [0]:
df_emp.write.format("delta").mode("overwrite").save("/Volumes/workspace/default/emp/Badrec/emp_prac_delta")


**DUPLICATES**

In [0]:
dbutils.fs.put('/Volumes/workspace/default/emp/duplicates.csv',"""id,name,loc,updated_date
1,ravi,bangalore,2021-01-01
1,ravi,chennai,2022-02-02
1,ravi,Hyderabad,2022-06-10
2,Raj,bangalore,2021-01-01
2,Raj,chennai,2022-02-02
3,Raj,Hyderabad,2022-06-10
4,Prasad,bangalore,2021-01-01
5,Mahesh,chennai,2022-02-02
4,Prasad,Hyderabad,2022-06-10
""")

In [0]:
df_dup = spark.read.format("csv").option("inferScheama",True).option("header", True).load("/Volumes/workspace/default/emp/duplicates.csv")
display(df_dup)
df_dup.write.format("delta").mode("overwrite").save("/Volumes/workspace/default/emp/dup/dup_data")

In [0]:
from pyspark.sql.functions import *
from delta.tables import *
from pyspark.sql.types import *
df_correct = df_dup.orderBy(desc("updated_date")).dropDuplicates(["id"])
df_correct.display()



from pyspark.sql.functions import *
from pyspark.sql.window import Window
from delta.tables import *
from pyspark.sql.types import *

df_correct_rn = Window.partitionBy("id").orderBy(desc("updated_date"))
df_rn = df_dup.withColumn("Row_num", row_number().over(df_correct_rn))
df_new =df_rn.filter(col("Row_num") == 1)
df_new.display()

**ADD FILE NAME COLUMN**

input_file_name()

In [0]:
from pyspark.sql.functions import input_file_name
from delta.tables import *
from pyspark.sql.types import *

In [0]:
filename_col = spark.read.format("csv").option("inferSchema",True).option("header",True).load("/Volumes/workspace/default/emp/duplicates.csv")
new_dup = filename_col.withColumn("filename",input_file_name())
new_dup.display()

In [0]:
from pyspark.sql.functions import col

filename_col = spark.read.format("csv").option("inferSchema", True).option("header", True).load("/Volumes/workspace/default/emp/duplicates.csv")
new_dup = filename_col.withColumn("filename", col("_metadata.file_path"))
emp_str = new_dup.withColumn(("filename",expr("substr(filename,1)")))
display(emp_str)

In [0]:
from pyspark.sql.functions import col, expr

filename_col = spark.read.format("csv").option("inferSchema", True).option("header", True).load("/Volumes/workspace/default/emp/duplicates.csv")
new_dup = filename_col.withColumn("filename", col("_metadata.file_path"))
emp_str = new_dup.withColumn("filename", expr("substr(filename, 37)"))
display(emp_str)

In [0]:
from pyspark.sql.functions import col, expr

filename_col = spark.read.format("csv").option("inferSchema", True).option("header", True).load("/Volumes/workspace/default/emp/duplicates.csv")
new_dup = filename_col.withColumn("filename", col("_metadata.file_path"))
#emp_str = new_dup.withColumn("filename", expr("substr(filename, instr(filename,'/',1,5))"))
display(new_dup)


In [0]:
from pyspark.sql.functions import col, expr

filename_col = spark.read.format("csv").option("inferSchema", True).option("header", True).load("/Volumes/workspace/default/emp/duplicates.csv")
new_dup = filename_col.withColumn("filename", col("_metadata.file_path"))
emp_str = new_dup.withColumn("filename", expr("substr(filename, instr(filename,'/',-1)+1)"))
display(new_dup)

In [0]:
dbutils.fs.put("/Volumes/workspace/default/emp/emp_data_date.csv","""EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
7369,SMITH,CLERK,7902,17-12-80,800,null,20
7499,ALLEN,SALESMAN,7698,20-02-81,1600,300,30
7521,WARD,SALESMAN,7698,22-02-81,1250,500,30
7566,JONES,MANAGER,7839,04-02-81,2975,null,20
7654,MARTIN,SALESMAN,7698,21-09-81,1250,1400,30
7698,SGR,MANAGER,7839,05-01-81,2850,null,30
7782,RAVI,MANAGER,7839,06-09-81,2450,null,10
7788,SCOTT,ANALYST,7566,19-04-87,3000,null,20
7839,KING,PRESIDENT,null,01-11-81,5000,null,10
7844,TURNER,SALESMAN,7698,09-08-81,1500,0,30
7876,ADAMS,CLERK,7788,23-05-87,1100,null,20
7900,JAMES,CLERK,7698,12-03-81,950,null,30
7902,FORD,ANALYST,7566,12-03-81,3000,null,20
7934,MILLER,CLERK,7782,01-03-82,1300,null,10
1234,SEKHAR,doctor,7777,31-12-99,667,78,80""")
     